# testscript.ipynb

A Jupyter notebook for testing and development of the AmBIENCe2ABM module.
First, let's simply impor the module.

In [ ]:
import ambience2abm as amb

## Read the raw data and assumptions

The raw data and assumptions are handled using the `AmBIENCeDataset` object,
with the constructor requiring values for the assumed `interior_node_depth` and `period_of_variations` *(explained in the docstring)*.

The following cells demonstrate/test functionality of the `AmBIENCeDataset`.

In [ ]:
### Read the raw data and assumptions.

ambience = amb.AmBIENCeDataset(
    interior_node_depth=0.1,
    period_of_variations=2225140,
)
ambience.data

In [ ]:
### Create unique building periods

building_periods = ambience.building_periods()
building_periods

In [ ]:
### Check the building type to stock mappings.

ambience.building_type_mappings

In [ ]:
### Check the building stock statistics

ambience.building_stock_statistics

In [ ]:
### Check structure type assumptions

ambience.structure_types

In [ ]:
### Calculate the structure statistics

ambience.calculate_structure_statistics()

In [ ]:
### Check fenestration assumptions.

ambience.fenestration

In [ ]:
### Check ventilation assumptions.

ambience.ventilation

In [ ]:
### Calculate ventilation and fenestration statistics

ambience.calculate_ventilation_and_fenestration_statistics()

## Process the full ArchetypeBuildingModel.jl dataset

The `ABMDataset` object contains the final processed data compatible with `ArchetypeBuildingModel.jl`,
as well as functions for exporting the Data Package containing said processed data.
The `ABMDataset` objects are constructed based on the raw `AmBIENCeDataset` object.


In [ ]:
### Process the full ABM Dataset

abmdata = amb.ABMDataset(ambience)
abmdata

In [ ]:
### Inspect building periods

abmdata.building_period

In [ ]:
### Inspect building stocks

abmdata.building_stock

In [ ]:
### Inspect structure types

abmdata.structure_type

In [ ]:
### Inspect building stock statistics

abmdata.building_stock_statistics

In [ ]:
### Inspect structure statistics

abmdata.structure_statistics

In [ ]:
### Inspect ventilation and fenestration statistics

abmdata.ventilation_and_fenestration_statistics

In [ ]:
### Try exporting to csvs

abmdata.export_csvs()

In [ ]:
### Try creating the datapackage

pkg = abmdata.create_datapackage()
pkg

## Check processed data against the original.

Let's do a few comparisons to try and see that the data processing is performing as intended.
For starters, let's make sure that aggregating over the `BUILDING MATERIAL COMBINATION CODE`
and disaggregating over the `HEATING SYSTEM PREVALENCY ON BUILDING STOCK` haven't distorted the
total heated floor area.

In [ ]:
### Calculate equally aggregated numbers of buildings both from the original data and the processed output.

# Declare cols to aggregate over
cols = [
    "building_type",
    "building_period",
    "location_id"
]

# Renaming and aggregation of original data.
ambience_total_numbers = ambience.data.rename(
    columns={
        "REFERENCE BUILDING USE CODE": "building_type",
        "REFERENCE BUILDING COUNTRY CODE": "location_id",
        "NUMBER OF REFERENCE BUILDINGS IN THE BUILDING STOCK SEGMENT": "number_of_buildings"
    }
).groupby(cols).agg({"number_of_buildings": "sum"})

# Aggregation of processed data.
abm_total_numbers = abmdata.building_stock_statistics.reset_index().groupby(cols).agg({"number_of_buildings": "sum"})

# Check the differences (neglecting near-floating-point level)
num_diff = ambience_total_numbers - abm_total_numbers
num_diff[num_diff["number_of_buildings"].abs() > 1e-6]

Which should be empty.
There was a bug in the raw heatsys data for `DE-OTH-2011-2021`
which required normalizing the heating system prevalencies.